# Memory for the chat model 

当你与那些语言模型进行交互的时候，他们不会记得你之前和他进行的交流内容，这在我们构建一些应用程序（如聊天机器人）的时候，是一个很大的问题 -- 显得不够智能！因此，在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。

当使用 LangChain 中的储存(Memory)模块时，它可以帮助保存和管理历史聊天消息，以及构建关于特定实体的知识。这些组件可以跨多轮对话储存信息，并允许在对话期间跟踪特定信息和上下文。

LangChain 提供了多种储存类型。其中，缓冲区储存允许保留最近的聊天消息，摘要储存则提供了对整个对话的摘要。实体储存 则允许在多轮对话中保留有关特定实体的信息。这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。储存模块可以通过简单的API调用来访问和更新，允许开发人员更轻松地实现对话历史记录的管理和维护。

此次课程主要介绍其中四种储存模块，其他模块可查看文档学习。
- 对话缓存储存 (ConversationBufferMemory）
- 对话缓存窗口储存 (ConversationBufferWindowMemory）
- 对话令牌缓存储存 (ConversationTokenBufferMemory）
- 对话摘要缓存储存 (ConversationSummaryBufferMemory）

在LangChain中，储存 指的是大语言模型（LLM）的短期记忆。为什么是短期记忆？那是因为LLM训练好之后 (获得了一些长期记忆)，它的参数便不会因为用户的输入而发生改变。当用户与训练好的LLM进行对话时，LLM会暂时记住用户的输入和它已经生成的输出，以便预测之后的输出，而模型输出完毕后，它便会“遗忘”之前用户的输入和它的输出。因此，之前的这些信息只能称作为LLM的短期记忆。  
  
为了延长LLM短期记忆的保留时间，则需要借助一些外部储存方式来进行记忆，以便在用户与LLM对话中，LLM能够尽可能的知道用户与它所进行的历史对话信息。 

In [2]:
# set the init envs 
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.8,
    model_name="gpt-3.5-turbo",
)
chat = ChatOpenAI(
    temperature=0.0,
    model_name = "gpt-3.5-turbo",
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI(
    model_name = "gpt-3.5-turbo",
    temperature=0.0
    )
print(chat)

llm = OpenAI(
    model_name = "gpt-3.5-turbo",
    temperature = 0.0
)
print(llm)

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='gpt-3.5-turbo' temperature=0.0 model_kwargs={} openai_api_key='sk-S0TKP0Sxk8H8VirL7a912cAf06314e46AfB0348eD40183B5' openai_api_base='https://openkey.cloud/v1' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=None
OpenAIChat
Params: {'model_name': 'gpt-3.5-turbo', 'temperature': 0.0}


d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
d:\Anaconda3\envs\LLM\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## 二、对话缓存储存
#### 2.1.1 初始化对话模型
  当我们运行预测(<font style="font-family:verdana; font-size:13.5px;color:#0055AA">predict</font>)时，生成了一些提示，如下所见，他说“以下是人类和AI之间友好的对话，AI健谈“等等，这实际上是LangChain生成的提示，以使系统进行希望和友好的对话，并且必须保存对话，并提示了当前已完成的模型链。

In [4]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory()

# 新建一个 ConversationChain Class 实例
# verbose参数设置为True时，程序会输出更详细的信息，以提供更多的调试或运行时信息。
# 相反，当将verbose参数设置为False时，程序会以更简洁的方式运行，只输出关键的信息。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

In [5]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

Langchaing 使用memory 来记录chatbot 已有的对话。

In [8]:
print(memory.buffer)
print()
print(memory.load_memory_variables({}))

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}


##### 用户也可以直接添加内容到储存缓存

In [10]:
memory = ConversationBufferMemory()  # 新建一个空的对话缓存记忆

memory.save_context({"input": "Hi"}, {"output": "What's up"})  # 向缓存区添加指定对话的输入输出

print(memory.buffer)
print()
print(memory.load_memory_variables({}))
print()
print()

memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

print(memory.buffer)
print()
print(memory.load_memory_variables({}))


Human: Hi
AI: What's up

{'history': "Human: Hi\nAI: What's up"}


Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}


当我们在使用大型语言模型进行聊天对话时，**大型语言模型本身实际上是无状态的。语言模型本身并不记得到目前为止的历史对话**。每次调用API结点都是独立的。储存(Memory)可以储存到目前为止的所有术语或对话，并将其输入或附加上下文到LLM中用于生成输出。如此看起来就好像它在进行下一轮对话的时候，记得之前说过什么。


## 三、对话缓存窗口储存
  
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵,这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。
  
针对以上问题，LangChain也提供了几种方便的储存方式来保存历史对话。其中，对话缓存窗口储存只保留一个窗口大小的对话。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大

In [13]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 1)


memory.save_context({"input": "Hi"}, {"output": "What's up"})  # 向缓存区添加指定对话的输入输出
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

print(memory.buffer)
print()
print(memory.load_memory_variables({}))


Human: Not much, just hanging
AI: Cool

{'history': 'Human: Not much, just hanging\nAI: Cool'}


In [19]:
ShortMemoryConversationChain = ConversationChain(llm = llm, 
                                                 memory = memory,
                                                 verbose=True)



In [20]:
ShortMemoryConversationChain.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How are you doing today?
Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How are you doing today?"

In [21]:
ShortMemoryConversationChain.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How are you doing today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [22]:
ShortMemoryConversationChain.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

## 四、对话token缓存储存

使用对话token缓存记忆，内存将限制保存的token数量。如果token数量超出指定数目，它会切掉这个对话的早期部分
以保留与最近的交流相对应的token数量，但不超过token限制。

### 4.3 补充

ChatGPT使用一种基于字节对编码（Byte Pair Encoding，BPE）的方法来进行tokenization（将输入文本拆分为token）。BPE是一种常见的tokenization技术，它将输入文本分割成较小的子词单元。 

OpenAI在其官方GitHub上公开了一个最新的开源Python库 [tiktoken](https://github.com/openai/tiktoken)，这个库主要是用来计算tokens数量的。相比较HuggingFace的tokenizer，其速度提升了好几倍。

具体token计算方式,特别是汉字和英文单词的token区别，具体课参考[知乎文章](https://www.zhihu.com/question/594159910) 。

In [26]:
from langchain.memory import ConversationTokenBufferMemory


memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## 五、对话摘要缓存储存

对话摘要缓存储存，**使用LLM编写到目前为止历史对话的摘要**，并将其保存

In [29]:
from langchain.memory import ConversationSummaryBufferMemory


# 创建一个长字符串
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# 使用对话摘要缓存记忆
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=400) 
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})
memory.save_context(
    {"input": "What is on the schedule today?"}, {"output": f"{schedule}"}
)

print(memory.load_memory_variables({})['history'])

System: The human greets the AI and asks what is on the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.


In [30]:
conversation = ConversationChain(llm=llm, 
                                 memory=memory, 
                                 verbose=True)

conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what is on the schedule for the day.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


"A good demo to show would be the Language Model API. It showcases the capabilities of LangChain by generating coherent and contextually relevant text based on user input. You can demonstrate how it can be used for various applications such as chatbots, content generation, and language translation. Additionally, you can highlight the model's ability to understand and respond to complex queries and provide accurate and informative answers."

In [31]:
print(memory.load_memory_variables({})['history'])

System: The human greets the AI and asks what is on the schedule for the day. The AI informs the human that there is a meeting at 8am with the product team, where a PowerPoint presentation needs to be prepared. From 9am to 12pm, the human has time to work on the LangChain project, which will be efficient due to the power of LangChain. At noon, the human has a lunch appointment with a customer who is traveling from over an hour away to learn about the latest advancements in AI. The AI advises the human to bring their laptop to showcase the latest LLM demo.
Human: What would be a good demo to show?
AI: A good demo to show would be the Language Model API. It showcases the capabilities of LangChain by generating coherent and contextually relevant text based on user input. You can demonstrate how it can be used for various applications such as chatbots, content generation, and language translation. Additionally, you can highlight the model's ability to understand and respond to complex quer